<a href="https://colab.research.google.com/github/meitalhp/DataSet_project/blob/main/predictions_data_unified_incl_flight.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [103]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import norm
import seaborn as sns
import datetime

In [104]:
entered_df=pd.read_csv('/content/clean_data_entered.csv')
not_entered_df=pd.read_csv('/content/clean_data_not_entered.csv')
flight_df=pd.read_csv('/content/clean_flight_result.csv')

In [105]:
# Merge the DataFrames on 'date', 'hour', and 'minute'
merged_df = pd.merge(entered_df, not_entered_df[['date', 'hour', 'minute', 'Average_Minutes_to_Order']],
                     on=['date', 'hour', 'minute'], how='left')

# Check the result
merged_df=merged_df.dropna(how='any')


In [106]:
# Merging flight data with grouped_df_enter_shift data
merged_df_w_flight = pd.merge(merged_df, flight_df[['date', 'hour', 'minute', 'Average_num_of_flights_T3']],
                             on=['date', 'hour', 'minute'], how='left')
merged_df_w_flight

,start_time,date,hour,minute,Average_Minutes_to_wait,weekday,day,month,Average_Minutes_to_Order,Average_num_of_flights_T3
0,2022-09-01 00:00:00,2022-09-01,0,0,38.884615,Thursday,1,September,33.375000,2.0
1,2022-09-01 00:15:00,2022-09-01,0,15,37.666667,Thursday,1,September,53.850000,NaN
2,2022-09-01 00:30:00,2022-09-01,0,30,30.040000,Thursday,1,September,35.562500,7.0
3,2022-09-01 00:45:00,2022-09-01,0,45,27.952381,Thursday,1,September,68.750000,NaN
4,2022-09-01 01:00:00,2022-09-01,1,0,21.761905,Thursday,1,September,53.562500,1.0
...,...,...,...,...,...,...,...,...,...,...
13972,2023-02-04 01:30:00,2023-02-04,1,30,56.166667,Saturday,4,February,28.400000,1.0
13973,2023-02-04 01:45:00,2023-02-04,1,45,56.500000,Saturday,4,February,51.333333,NaN
13974,2023-02-04 02:00:00,2023-02-04,2,0,52.333333,Saturday,4,February,39.714286,1.0
13975,2023-02-04 02:15:00,2023-02-04,2,15,41.333333,Saturday,4,February,71.666667,NaN


In [107]:
# Fill NULL values in Average_num_of_flights_T3
def fill_null_flights(df):
    df = df.copy()

    # Forward fill and backward fill the missing values
    df['Average_num_of_flights_T3'] = df['Average_num_of_flights_T3'].fillna(method='ffill')
    df['Average_num_of_flights_T3'] = df['Average_num_of_flights_T3'].fillna(method='bfill')

    # Divide the values by 2 to distribute over 15-minute intervals
    df['Average_num_of_flights_T3'] = df['Average_num_of_flights_T3'] / 2

    return df
#apply the func on the dfs
merged_df_w_flight=fill_null_flights(merged_df_w_flight)
merged_df_w_flight

,start_time,date,hour,minute,Average_Minutes_to_wait,weekday,day,month,Average_Minutes_to_Order,Average_num_of_flights_T3
0,2022-09-01 00:00:00,2022-09-01,0,0,38.884615,Thursday,1,September,33.375000,1.0
1,2022-09-01 00:15:00,2022-09-01,0,15,37.666667,Thursday,1,September,53.850000,1.0
2,2022-09-01 00:30:00,2022-09-01,0,30,30.040000,Thursday,1,September,35.562500,3.5
3,2022-09-01 00:45:00,2022-09-01,0,45,27.952381,Thursday,1,September,68.750000,3.5
4,2022-09-01 01:00:00,2022-09-01,1,0,21.761905,Thursday,1,September,53.562500,0.5
...,...,...,...,...,...,...,...,...,...,...
13972,2023-02-04 01:30:00,2023-02-04,1,30,56.166667,Saturday,4,February,28.400000,0.5
13973,2023-02-04 01:45:00,2023-02-04,1,45,56.500000,Saturday,4,February,51.333333,0.5
13974,2023-02-04 02:00:00,2023-02-04,2,0,52.333333,Saturday,4,February,39.714286,0.5
13975,2023-02-04 02:15:00,2023-02-04,2,15,41.333333,Saturday,4,February,71.666667,0.5


In [108]:
def add_multiple_previous_observations(df, target_column, shifts):

    # Add the specified previous observations
    for n in shifts:
        df[f'{target_column}_t-{n}'] = df[target_column].shift(n)

    return df



In [109]:
shifts = list(range(1,25))
grouped_df_shift= add_multiple_previous_observations(df=merged_df_w_flight,target_column='Average_Minutes_to_Order',shifts=shifts)
grouped_df_shift.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13977 entries, 0 to 13976
Data columns (total 34 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   start_time                     13977 non-null  object 
 1   date                           13977 non-null  object 
 2   hour                           13977 non-null  int64  
 3   minute                         13977 non-null  int64  
 4   Average_Minutes_to_wait        13977 non-null  float64
 5   weekday                        13977 non-null  object 
 6   day                            13977 non-null  int64  
 7   month                          13977 non-null  object 
 8   Average_Minutes_to_Order       13977 non-null  float64
 9   Average_num_of_flights_T3      13977 non-null  float64
 10  Average_Minutes_to_Order_t-1   13976 non-null  float64
 11  Average_Minutes_to_Order_t-2   13975 non-null  float64
 12  Average_Minutes_to_Order_t-3   13974 non-null 

In [110]:
def add_obs(df, target_column, shifts):
    """
    Adds new columns with specified previous observations for a given target column.

    :param df: DataFrame containing the data
    :param target_column: The name of the target column to add previous observations for
    :param shifts: A list of tuples where each tuple contains the number of previous observations (n) and the frequency (freq)
                   e.g., [(5, '1D'), (8, '15T')]
    :return: DataFrame with new columns added
    """
    df = df.copy()

    # Ensure the data is sorted by datetime
    df = df.sort_values(by='date')

    # Set datetime as the index
    df.set_index('datetime', inplace=True)

    # Add the specified previous observations
    for n, freq in shifts:
        df[f'{target_column}_t-{n}_{freq}'] = df[target_column].shift(n, freq=freq)

    # Reset the index to make datetime a regular column again
    df.reset_index(drop=True, inplace=True)

    return df

In [111]:
def add_is_weekend(df, date_column):
    """
    Adds a binary column indicating if the date is a weekend in Israel.

    :param df: DataFrame containing the data
    :param date_column: The name of the column containing the date
    :return: DataFrame with the new 'is_weekend' column added
    """

    # Add is_weekend column
    df['is_weekend'] = df[date_column].isin(['Friday','Saturday']).astype(int)
    return df

In [112]:
import holidays
def add_is_holiday(df, date_column):
    """
    Adds a binary column indicating if the date is an Israeli holiday.

    :param df: DataFrame containing the data
    :param date_column: The name of the column containing the date
    :return: DataFrame with the new 'is_holiday' column added
    """
    # Ensure the date column is in datetime format
    df[date_column] = pd.to_datetime(df[date_column])

    # Get the list of holidays in Israel
    israel_holidays = holidays.Israel()

    # Add is_holiday column
    df['is_holiday'] = df[date_column].apply(lambda x: 1 if x in israel_holidays else 0)
    return df

In [113]:

grouped_df_shift=add_is_weekend(grouped_df_shift,'weekday')
grouped_df_shift=add_is_holiday(grouped_df_shift,'date')

In [114]:
# verify 1 for יום כיפור
grouped_df_shift[grouped_df_shift['date']=='2022-10-05']

,start_time,date,hour,minute,Average_Minutes_to_wait,weekday,day,month,Average_Minutes_to_Order,Average_num_of_flights_T3,...,Average_Minutes_to_Order_t-17,Average_Minutes_to_Order_t-18,Average_Minutes_to_Order_t-19,Average_Minutes_to_Order_t-20,Average_Minutes_to_Order_t-21,Average_Minutes_to_Order_t-22,Average_Minutes_to_Order_t-23,Average_Minutes_to_Order_t-24,is_weekend,is_holiday
3122,2022-10-05 21:15:00,2022-10-05,21,15,95.000000,Wednesday,5,October,92.000000,1.0,...,82.090909,67.166667,62.736842,68.111111,63.857143,62.565217,69.115385,51.240000,0,1
3123,2022-10-05 21:30:00,2022-10-05,21,30,89.200000,Wednesday,5,October,69.250000,0.0,...,94.357143,82.090909,67.166667,62.736842,68.111111,63.857143,62.565217,69.115385,0,1
3124,2022-10-05 21:45:00,2022-10-05,21,45,89.416667,Wednesday,5,October,54.250000,0.0,...,60.000000,94.357143,82.090909,67.166667,62.736842,68.111111,63.857143,62.565217,0,1
3125,2022-10-05 22:00:00,2022-10-05,22,0,78.615385,Wednesday,5,October,50.400000,1.5,...,46.100000,60.000000,94.357143,82.090909,67.166667,62.736842,68.111111,63.857143,0,1
3126,2022-10-05 22:15:00,2022-10-05,22,15,68.850000,Wednesday,5,October,69.375000,1.5,...,50.200000,46.100000,60.000000,94.357143,82.090909,67.166667,62.736842,68.111111,0,1
3127,2022-10-05 22:30:00,2022-10-05,22,30,60.461538,Wednesday,5,October,51.454545,3.5,...,82.666667,50.200000,46.100000,60.000000,94.357143,82.090909,67.166667,62.736842,0,1
3128,2022-10-05 22:45:00,2022-10-05,22,45,58.750000,Wednesday,5,October,53.785714,3.5,...,107.000000,82.666667,50.200000,46.100000,60.000000,94.357143,82.090909,67.166667,0,1
3129,2022-10-05 23:00:00,2022-10-05,23,0,56.000000,Wednesday,5,October,75.333333,2.5,...,43.666667,107.000000,82.666667,50.200000,46.100000,60.000000,94.357143,82.090909,0,1
3130,2022-10-05 23:15:00,2022-10-05,23,15,42.000000,Wednesday,5,October,58.136364,2.5,...,79.000000,43.666667,107.000000,82.666667,50.200000,46.100000,60.000000,94.357143,0,1
3131,2022-10-05 23:30:00,2022-10-05,23,30,27.000000,Wednesday,5,October,68.750000,2.5,...,112.500000,79.000000,43.666667,107.000000,82.666667,50.200000,46.100000,60.000000,0,1
